# CC3092 — Deep Learning – Hoja de Trabajo 2

Integrantes: 
Diego Valenzuela - 22309
Gerson Ramirez - 22281

In [1]:
import torch
print("torch version:", torch.__version__)
print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    x = torch.rand((2, 2), device="cuda")
    print("Tensor en dispositivo:", x.device)


torch version: 2.8.0+cu126
CUDA disponible: True
GPU: NVIDIA GeForce GTX 1660 Ti with Max-Q Design
Tensor en dispositivo: cuda:0


In [2]:
import time, math, random
from dataclasses import dataclass
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.use_deterministic_algorithms(False)  
set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

## Task 1 — Carga de Iris + split (train/val) + escalado

Se usa Iris para clasificación multiclase (3 clases). Se separa 75/25 (train/val) con estratificación para mantener proporciones de clases. Se estandarizan atributos (media 0, var 1), lo que acelera y estabiliza el entrenamiento del MLP.

In [3]:
iris = load_iris()
X = iris.data.astype(np.float32)
y = iris.target.astype(np.int64)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train).astype(np.float32)
X_val   = scaler.transform(X_val).astype(np.float32)

X_train_t = torch.from_numpy(X_train)
y_train_t = torch.from_numpy(y_train)
X_val_t   = torch.from_numpy(X_val)
y_val_t   = torch.from_numpy(y_val)

train_ds = TensorDataset(X_train_t, y_train_t)
val_ds   = TensorDataset(X_val_t,   y_val_t)

len(train_ds), len(val_ds)


(112, 38)

## Task 2 — MLP simple y parametrizable

Arquitectura feedforward con capas ocultas configurables, activación seleccionable y Dropout (para Task 4). La capa final entrega logits (sin softmax); la función de pérdida se encarga de lo demás.

In [4]:
class MLP(nn.Module):
    def __init__(self, in_dim: int = 4, hidden: List[int] = [32, 16],
                 out_dim: int = 3, activation: str = "relu", dropout_p: float = 0.0):
        super().__init__()
        acts = {
            "relu": nn.ReLU(),
            "tanh": nn.Tanh(),
            "gelu": nn.GELU(),
            "leakyrelu": nn.LeakyReLU(0.1),
        }
        self.act = acts.get(activation.lower(), nn.ReLU())
        layers = []
        prev = in_dim
        for h in hidden:
            layers += [nn.Linear(prev, h), self.act]
            if dropout_p and dropout_p > 0.0:
                layers += [nn.Dropout(dropout_p)]
            prev = h
        layers += [nn.Linear(prev, out_dim)]
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


## 5) Task 3 — Funciones de pérdida (CE, NLL, MSE) parametrizadas

Usaremos CrossEntropyLoss, NLLLoss (con log_softmax) y MSE (con one-hot + softmax). Esto cumple el requisito de ≥3 pérdidas y permite comparar convergencia y rendimiento

In [5]:
@dataclass
class LossAdapter:
    name: str
    criterion: nn.Module
    def __call__(self, logits: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        if self.name == "cross_entropy":
            return self.criterion(logits, y)
        elif self.name == "nll":
            return self.criterion(F.log_softmax(logits, dim=1), y)
        elif self.name == "mse":
            probs = F.softmax(logits, dim=1)
            one_hot = F.one_hot(y, num_classes=logits.shape[1]).float()
            return self.criterion(probs, one_hot)
        else:
            raise ValueError(f"Pérdida desconocida: {self.name}")

def make_loss(name: str) -> LossAdapter:
    name = name.lower()
    if name in ("crossentropy", "ce", "cross_entropy"):
        return LossAdapter("cross_entropy", nn.CrossEntropyLoss())
    if name in ("nll", "nllloss"):
        return LossAdapter("nll", nn.NLLLoss())
    if name in ("mse", "mseloss"):
        return LossAdapter("mse", nn.MSELoss())
    raise ValueError(f"Pérdida no soportada: {name}")


## Task 4 — Regularización (L1, L2, Dropout)

* L2: se usa como weight_decay del optimizador.
* L1: se suma manualmente a la pérdida: λ₁ * Σ|w|.
* Dropout: parametrizado en el modelo (ya incluido).

In [6]:
def l1_penalty(model: nn.Module) -> torch.Tensor:
    total = torch.tensor(0., device=device)
    for p in model.parameters():
        if p.requires_grad:
            total = total + p.abs().sum()
    return total


## Task 5 — “Algoritmos de optimización”

* Batch GD: actualiza con todo el conjunto de entrenamiento (batch único).
* Mini-Batch GD: actualiza por lotes pequeños (p. ej. 16).
* SGD: actualiza por cada muestra (batch_size=1).

> Usaremos el mismo optimizador (torch.optim.SGD) pero cambiaremos el tamaño del batch para reflejar cada técnica.

In [7]:
def make_loader(dataset, mode: str, batch_size: int = 16, shuffle: bool = True):
    mode = mode.lower()
    if mode == "batch_gd":
        bs = len(dataset)
    elif mode == "sgd":
        bs = 1
    elif mode == "mini-batch" or mode == "mini_batch":
        bs = batch_size
    else:
        raise ValueError("mode debe ser 'batch_gd', 'mini-batch' o 'sgd'")
    return DataLoader(dataset, batch_size=bs, shuffle=shuffle)


# Parte 2

1. **¿Cuál es la principal innovación de la arquitectura Transformer?**

  La gran innovación del Transformer es eliminar por completo las recurrencias y convoluciones. En su lugar, se basa únicamente en mecanismos de atención, especialmente *self-attention*, para modelar dependencias entre tokens. Esto permite mayor paralelización en el entrenamiento y mejor manejo de dependencias largas.

2. **¿Cómo funciona el mecanismo de atención del scaled dot-product?**

  El scaled dot-product attention toma consultas (Q), claves (K) y valores (V). Calcula los productos punto entre las consultas y todas las claves, los escala por $1/\sqrt{d_k}$ para evitar gradientes muy pequeños, y aplica softmax para obtener pesos de atención. Luego usa esos pesos para combinar linealmente los valores:

  $$
  \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
  $$

  Este escalado es lo que diferencia al mecanismo y lo hace más estable.

3. **¿Por qué se utiliza la atención de múltiples cabezales en Transformer?**

  La multi-head attention proyecta las Q, K y V en distintos subespacios, aplica atención en paralelo y concatena los resultados. Esto permite que el modelo aprenda a atender a diferentes aspectos de la información en paralelo, como relaciones sintácticas y semánticas distintas. Con un único “head”, la información se promediaría y se perderían matices
